In [ ]:
import vectorbt as vbt

import pandas as pd
import numpy as np
import yaml
from typing import Optional
from datetime import datetime, time

with open('config.yaml', 'r') as fp:
    config = yaml.safe_load(fp)

"""1. Import Config"""
DATA_FP = config['DATA_FP']
DATA_TF = config['DATA_TF']
DATA_1MIN_FP = config['DATA_1MIN_FP']

FEES = config['FEES']
SLIPPAGE = config['SLIPPAGE']
SIZE = config['SIZE']
SIZE_TYPE = config['SIZE_TYPE']
INIT_BALANCE = config['INIT_BALANCE']
BACKTESTING_DATES_START = config['BACKTESTING_DATES']['START']
BACKTESTING_DATES_END = config['BACKTESTING_DATES']['END']
WICK_RATIO = config['WICK_RATIO']
OP_WICK_RATIO = config['OP_WICK_RATIO']
RR = config['RR']
TRADING_TIME_START = config['TRADING_TIME']['START']
TRADING_TIME_END = config['TRADING_TIME']['END']

"""2. Read and process data."""
df_custom = pd.read_csv(DATA_FP)
df_custom = df_custom.set_index('Time')
df_custom.index = pd.to_datetime(df_custom.index)
df_custom = df_custom[BACKTESTING_DATES_START: BACKTESTING_DATES_END] # TODO

df_1min = pd.read_csv(DATA_1MIN_FP)
df_1min = df_1min.set_index('Time')
df_1min.index = pd.to_datetime(df_1min.index)

"""3. Mark signals, SL, TP on custom timeframe. """
df_custom['Direction'] = -1
df_custom.loc[df_custom['Close'] > df_custom['Open'], 'Direction'] = 1
df_custom.loc[df_custom['Direction'] == 1, 'UWL'] = df_custom['High'] - df_custom['Close']
df_custom.loc[df_custom['Direction'] == -1, 'UWL'] = df_custom['High'] - df_custom['Open']
df_custom.loc[df_custom['Direction'] == 1, 'LWL'] = df_custom['Open'] - df_custom['Low']
df_custom.loc[df_custom['Direction'] == -1, 'LWL'] = df_custom['Close'] - df_custom['Low']
df_custom['Body'] = (df_custom['Close'] - df_custom['Open']).abs()

bullish_entry_mask = (
    (df_custom['LWL'] >= WICK_RATIO * df_custom['Body']) &
    (df_custom['Direction'] == 1) &
    (df_custom['UWL'] * OP_WICK_RATIO <= df_custom['LWL'])
)

bearish_entry_mask = (
    (df_custom['UWL'] >= WICK_RATIO * df_custom['Body']) &
    (df_custom['Direction'] == -1) &
    (df_custom['LWL'] * OP_WICK_RATIO <= df_custom['UWL'])
)

df_custom['Bullish Entry'] = bullish_entry_mask
df_custom['Bearish Entry'] = bearish_entry_mask

df_custom = df_custom.drop(columns=['UWL', 'LWL', 'Body', 'Direction'])
df_custom.loc[df_custom['Bullish Entry'] == True, 'SL'] = df_custom['Low']
df_custom.loc[df_custom['Bearish Entry'] == True, 'SL'] = df_custom['High']

df_custom.loc[df_custom['Bullish Entry'] == True, 'TP'] = df_custom['Close'] + (df_custom['Close'] - df_custom['Low']) * RR
df_custom.loc[df_custom['Bearish Entry'] == True, 'TP'] = df_custom['Close'] - (df_custom['High'] - df_custom['Close']) * RR

"""4. Reindex to 1min timeframe. """
df_1min['Bullish Entry'] = df_custom['Bullish Entry'].shift(1).reindex(df_1min.index, method='ffill').copy()
df_1min['Bearish Entry'] = df_custom['Bearish Entry'].shift(1).reindex(df_1min.index, method='ffill').copy()
df_1min['SL'] = df_custom['SL'].shift(1).reindex(df_1min.index, method='ffill').copy()
df_1min['TP'] = df_custom['TP'].shift(1).reindex(df_1min.index, method='ffill').copy()
df_1min['Date and Hour'] = df_1min.index.floor('h')
df_1min = df_1min.dropna()



In [ ]:
"""5. Main Loop"""

index_arr_1min = df_1min.index.to_numpy()
bullish_entry_arr_1min = df_1min['Bullish Entry'].to_numpy()
bearish_entry_arr_1min = df_1min['Bearish Entry'].to_numpy()
sl_arr_1min = df_1min['SL'].to_numpy()
tp_arr_1min = df_1min['TP'].to_numpy()
high_arr = df_1min['High'].to_numpy()
low_arr = df_1min['Low'].to_numpy()
close_arr = df_1min['Close'].to_numpy()
date_and_hour_arr = df_1min['Date and Hour'].to_numpy()
candle_time_arr = df_1min.index.time

price_arr = np.full(len(index_arr_1min), np.nan)

bullish_entries_arr = np.full(len(index_arr_1min), False)
bearish_entries_arr = np.full(len(index_arr_1min), False)

bullish_exit_arr = np.full(len(index_arr_1min), False)
bearish_exit_arr = np.full(len(index_arr_1min), False)

opened_trade_direction = None # bullish/bearish/None
current_sl = None; current_tp = None # float/None

opened_date_and_hour = {}

trading_start_time = time(hour=int(TRADING_TIME_START.split(':')[0]), minute=int(TRADING_TIME_START.split(':')[1]))
trading_end_time = time(hour=int(TRADING_TIME_END.split(':')[0]), minute=int(TRADING_TIME_END.split(':')[1]))

for i in range(len(index_arr_1min)):
    if opened_trade_direction is None: # if trade is not opened, check conditions to open
        date_and_hour = date_and_hour_arr[i]
        bullish_signal = bullish_entry_arr_1min[i]
        bearish_signal = bearish_entry_arr_1min[i]
        
        if (not bullish_signal and not bearish_signal) or opened_date_and_hour.get(date_and_hour) == True: # no entry
            continue

        if not (trading_start_time <= candle_time_arr[i] <= trading_end_time):
            continue

        opened_trade_direction = 'bullish' if bullish_signal else 'bearish' # bullish/bearish entry
        bullish_entries_arr[i] = True if bullish_signal else False
        bearish_entries_arr[i] = True if bearish_signal else False
        price_arr[i] = close_arr[i]
        current_sl, current_tp = sl_arr_1min[i], tp_arr_1min[i]
        opened_date_and_hour[date_and_hour] = True
        print(index_arr_1min[i], f'{opened_trade_direction} trade is opened on price: {price_arr[i]}, sl: {current_sl}, tp: {current_tp}')

        continue

    elif opened_trade_direction == 'bullish':
        close_ = False
        closing_price = None

        if low_arr[i] <= current_sl: # sl hit
            close_ = True
            closing_price = current_sl

        elif high_arr[i] >= current_tp: # tp hit
            close_ = True
            closing_price = current_tp

        
        if close_:
            price_arr[i] = closing_price
            print(index_arr_1min[i], f'{opened_trade_direction} trade is closed on price: {price_arr[i]}')
            bullish_exit_arr[i], opened_trade_direction, current_sl, current_tp = True, None, None, None
            

    elif opened_trade_direction == 'bearish':
        close_ = False
        closing_price = None

        if high_arr[i] >= current_sl: # sl hit
            close_ = True
            closing_price = current_sl

        elif low_arr[i] <= current_tp: # tp hit
            close_ = True
            closing_price = current_tp

        
        if close_:
            price_arr[i] = closing_price
            print(index_arr_1min[i], f'{opened_trade_direction} trade is closed on price: {price_arr[i]}')
            bearish_exit_arr[i], opened_trade_direction, current_sl, current_tp = True, None, None, None


"""5. Backtest with Vectorbt"""
pf = vbt.Portfolio.from_signals(
    close=df_1min['Close'],
    price=price_arr,
    entries=bullish_entries_arr,
    exits=bullish_exit_arr,
    short_entries=bearish_entries_arr,
    short_exits=bearish_exit_arr,
    size=SIZE,
    size_type=SIZE_TYPE,
    fees=FEES,
    slippage=SLIPPAGE,
    init_cash=INIT_BALANCE,
    freq='1min'
)


In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', '', '')))
from src.functions import save_backtesting_results

save_backtesting_results(pf)